In [ ]:
import numpy as np
import math

# Physical constants
ANGLE = 1.05  # degrees
THETA = ANGLE * math.pi / 180  # radians
V_F = 2.1354  # ℏv_F/a in eV
U1 = 0.0797  # interlayer coupling (eV)
U2 = 0.09  # interlayer coupling (eV)
A = 0.246  # lattice constant (nm)

# Lattice constants for graphene
A_X = np.sqrt(3)
A_Y = 1


def compute_moire_reciprocal_lattice(nx, ny, theta):
    """
    Compute the moiré reciprocal lattice vectors.
    
    Parameters:
    -----------
    nx, ny : int
        Number of lattice points in x and y directions
    theta : float
        Twist angle in radians
    
    Returns:
    --------
    G : list
        List of reciprocal lattice vectors
    """
    n = (nx + 1) * (ny + 1)
    
    # Moiré reciprocal lattice vector magnitude
    dG = [-4 * math.sin(theta / 2) * np.pi / A_X,
          -4 * math.sin(theta / 2) * np.pi / A_Y]
    
    G = []
    xlist = np.linspace(-nx / 2, nx / 2, nx + 1)
    ylist = np.linspace(-ny / 2, ny / 2, ny + 1)
    
    for i in xlist:
        for j in ylist:
            G.append([(i - 2 * j) * dG[0], i * dG[1]])
    
    return G


def compute_K_points(theta):
    """
    Compute K-points for both layers.
    
    Parameters:
    -----------
    theta : float
        Twist angle in radians
    
    Returns:
    --------
    K1, K2 : tuple
        K-points for layers 1 and 2 as (Kx, Ky)
    """
    K1x = -4 * np.pi * math.cos(theta / 2) / 3
    K1y = 4 * np.pi * math.sin(theta / 2) / 3
    
    K2x = -4 * np.pi * math.cos(theta / 2) / 3
    K2y = -4 * np.pi * math.sin(theta / 2) / 3
    
    return (K1x, K1y), (K2x, K2y)


def intralayer_hamiltonian(nx, ny, delta, k_vec, v=V_F, theta=THETA):
    """
    Construct the intralayer part of the Hamiltonian (H_1 and H_2).
    
    Parameters:
    -----------
    nx, ny : int
        Number of lattice points
    delta : float
        Sublattice potential (eV)
    k_vec : tuple
        Momentum vector (kx, ky)
    v : float
        Fermi velocity parameter
    theta : float
        Twist angle in radians
    
    Returns:
    --------
    H0 : ndarray
        Intralayer Hamiltonian matrix
    """
    n = (nx + 1) * (ny + 1)
    dim = 4 * n
    H0 = np.zeros((dim, dim), dtype=complex)
    
    # Get reciprocal lattice and K-points
    G = compute_moire_reciprocal_lattice(nx, ny, theta)
    K1, K2 = compute_K_points(theta)
    
    kx, ky = k_vec
    
    # Rotation factors
    cos_half = math.cos(theta / 2)
    sin_half = math.sin(theta / 2)
    rot_plus = cos_half - 1j * sin_half  # R(+θ/2)
    rot_minus = cos_half + 1j * sin_half  # R(-θ/2)
    
    # Fill the Hamiltonian matrix
    for l in range(nx + 1):
        for m in range(ny + 1):
            idx = l * (ny + 1) + m
            
            # Momentum relative to K-points
            q1x = kx + G[idx][0] - K1[0]
            q1y = ky + G[idx][1] - K1[1]
            q2x = kx + G[idx][0] - K2[0]
            q2y = ky + G[idx][1] - K2[1]
            
            # Layer 1: (A1, B1) block
            H0[idx, idx] = delta
            H0[idx, idx + n] = -v * rot_plus * (q1x - 1j * q1y)
            H0[idx + n, idx] = -v * rot_minus * (q1x + 1j * q1y)
            H0[idx + n, idx + n] = -delta
            
            # Layer 2: (A2, B2) block
            H0[idx + 2*n, idx + 2*n] = delta
            H0[idx + 2*n, idx + 3*n] = -v * rot_minus * (q2x - 1j * q2y)
            H0[idx + 3*n, idx + 2*n] = -v * rot_plus * (q2x + 1j * q2y)
            H0[idx + 3*n, idx + 3*n] = -delta
    
    return H0


def interlayer_hamiltonian(nx, ny, u1=U1, u2=U2):
    """
    Construct the interlayer coupling part of the Hamiltonian (U matrix).
    
    Parameters:
    -----------
    nx, ny : int
        Number of lattice points
    u1, u2 : float
        Interlayer coupling parameters (eV)
    
    Returns:
    --------
    H1 : ndarray
        Interlayer coupling Hamiltonian matrix
    """
    n = (nx + 1) * (ny + 1)
    dim = 4 * n
    H1 = np.zeros((dim, dim), dtype=complex)
    
    # Phase factors for the three coupling terms
    phase_pos = np.exp(1j * 2 * np.pi / 3)  # e^(i2π/3)
    phase_neg = np.exp(-1j * 2 * np.pi / 3)  # e^(-i2π/3)
    
    # First coupling term (on-site in moiré lattice)
    for l in range(nx + 1):
        for m in range(ny + 1):
            idx = l * (ny + 1) + m
            
            # Upper-right block (layer 1 to layer 2)
            H1[idx, idx + 2*n] = u1
            H1[idx, idx + 3*n] = u2
            H1[idx + n, idx + 2*n] = u2
            H1[idx + n, idx + 3*n] = u1
            
            # Lower-left block (layer 2 to layer 1, Hermitian conjugate)
            H1[idx + 2*n, idx] = u1
            H1[idx + 3*n, idx] = u2
            H1[idx + 2*n, idx + n] = u2
            H1[idx + 3*n, idx + n] = u1
    
    # Second coupling term (G_1^M direction)
    for l in range(nx):
        for m in range(ny + 1):
            idx1 = l * (ny + 1) + m
            idx2 = (l + 1) * (ny + 1) + m
            
            H1[idx1, idx2 + 2*n] = u1
            H1[idx1, idx2 + 3*n] = u2 * phase_neg
            H1[idx1 + n, idx2 + 2*n] = u2 * phase_pos
            H1[idx1 + n, idx2 + 3*n] = u1
            
            H1[idx2 + 2*n, idx1] = u1
            H1[idx2 + 3*n, idx1] = u2 * phase_pos
            H1[idx2 + 2*n, idx1 + n] = u2 * phase_neg
            H1[idx2 + 3*n, idx1 + n] = u1
    
    # Third coupling term (G_1^M + G_2^M direction)
    for l in range(1, nx + 1):
        for m in range(1, ny + 1):
            idx1 = (l - 1) * (ny + 1) + m - 1
            idx2 = l * (ny + 1) + m
            
            H1[idx1, idx2 + 2*n] = u1
            H1[idx1, idx2 + 3*n] = u2 * phase_pos
            H1[idx1 + n, idx2 + 2*n] = u2 * phase_neg
            H1[idx1 + n, idx2 + 3*n] = u1
            
            H1[idx2 + 2*n, idx1] = u1
            H1[idx2 + 3*n, idx1] = u2 * phase_neg
            H1[idx2 + 2*n, idx1 + n] = u2 * phase_pos
            H1[idx2 + 3*n, idx1 + n] = u1
    
    return H1


def tbg_hamiltonian(nx, ny, delta, k_vec, v=V_F, theta=THETA, u1=U1, u2=U2):
    """
    Construct the full TBG Hamiltonian.
    
    Parameters:
    -----------
    nx, ny : int
        Number of lattice points
    delta : float
        Sublattice potential (eV)
    k_vec : tuple
        Momentum vector (kx, ky)
    v : float
        Fermi velocity parameter
    theta : float
        Twist angle in radians
    u1, u2 : float
        Interlayer coupling parameters (eV)
    
    Returns:
    --------
    H : ndarray
        Full TBG Hamiltonian matrix
    """
    H0 = intralayer_hamiltonian(nx, ny, delta, k_vec, v, theta)
    H1 = interlayer_hamiltonian(nx, ny, u1, u2)
    
    return H0 + H1
